The imageprocessorapp is a drag-and-drop GUI tool using Tkinter and tkinterdnd2 that allows users to load an image, process its grayscale pixel data with multiple imputation methods (original, mean, median, constant, and one-hot encoded), and save all these results into a csv file

In [10]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image
import numpy as np
import pandas as pd
from tkinterdnd2 import DND_FILES, TkinterDnD

class ImageProcessorApp:
    def __init__(self, master):
        self.master = master
        master.title("Image to CSV Processor")

        self.image_path = None
        self.pixel_data = None

        # UI Setup
        self.drop_area = tk.Label(master, text="Drag and Drop Image Here\nor",
                                  bg="lightgray", width=50, height=10, relief="groove", borderwidth=2)
        self.drop_area.pack(pady=10, padx=10, fill="both", expand=True)

        self.drop_area.drop_target_register(DND_FILES)
        self.drop_area.dnd_bind('<<Drop>>', self.handle_drop)
        self.drop_area.dnd_bind('<<DragEnter>>', self.on_drag_enter)
        self.drop_area.dnd_bind('<<DragLeave>>', self.on_drag_leave)

        self.load_button = tk.Button(master, text="Browse for Image", command=self.load_image)
        self.load_button.pack(pady=5)

        self.constant_value_label = tk.Label(master, text="Constant Value (default 128):")
        self.constant_value_label.pack(pady=5)

        self.constant_value_input = tk.Entry(master)
        self.constant_value_input.insert(0, "128")
        self.constant_value_input.pack(pady=5)

        self.process_button = tk.Button(master, text="Process and Convert to CSV", command=self.process_and_save)
        self.process_button.pack(pady=10)

        self.status_label = tk.Label(master, text="", fg="blue")
        self.status_label.pack(pady=5)

    def load_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.gif;*.bmp")])
        if file_path:
            self._load_and_process_image(file_path)

    def handle_drop(self, event):
        file_paths = self.master.splitlist(event.data)
        if file_paths:
            file_path = file_paths[0].replace('{', '').replace('}', '')
            self._load_and_process_image(file_path)
        self.on_drag_leave(event)

    def _load_and_process_image(self, file_path):
        if not file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            messagebox.showerror("Invalid File", "Please drop or select a valid image file.")
            self.status_label.config(text="Invalid file type.", fg="red")
            return

        self.image_path = file_path
        try:
            img = Image.open(self.image_path).convert("L")
            self.pixel_data = np.array(img).flatten()
            self.status_label.config(text=f"Image loaded: {self.image_path.split('/')[-1]}", fg="green")
        except Exception as e:
            messagebox.showerror("Loading Error", f"Error loading image: {e}")
            self.status_label.config(text=f"Error loading image: {e}", fg="red")
            self.pixel_data = None

    def on_drag_enter(self, event):
        self.drop_area.config(bg="lightblue")

    def on_drag_leave(self, event):
        self.drop_area.config(bg="lightgray")

    def process_and_save(self):
        if self.pixel_data is None:
            messagebox.showwarning("No Image", "Please load an image first.")
            self.status_label.config(text="Please load an image first.", fg="orange")
            return

        try:
            constant_val = float(self.constant_value_input.get()) if self.constant_value_input.get() else 128.0
        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid numeric value for Constant.")
            self.status_label.config(text="Invalid constant value.", fg="red")
            return

        original = self.pixel_data
        mean_val = np.mean(original)
        median_val = np.median(original)

        mean_imputed = np.where(original == 0, mean_val, original)
        median_imputed = np.where(original == 0, median_val, original)
        constant_imputed = np.full_like(original, constant_val)
        encoded = original / 255.0

        df = pd.DataFrame({
            "type": ["original", "mean", "median", "constant", "encoded"],
            **{f"p{i}": [original[i], mean_imputed[i], median_imputed[i], constant_imputed[i], encoded[i]]
               for i in range(len(original))}
        })

        output_file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV Files", "*.csv")])
        if output_file_path:
            try:
                df.to_csv(output_file_path, index=False)
                messagebox.showinfo("Success", f"Processed data saved to: {output_file_path}")
                self.status_label.config(text=f"Data saved to: {output_file_path.split('/')[-1]}", fg="green")
            except Exception as e:
                messagebox.showerror("Saving Error", f"Error saving CSV: {e}")
                self.status_label.config(text=f"Error saving CSV: {e}", fg="red")

if __name__ == "__main__":
    root = TkinterDnD.Tk()
    app = ImageProcessorApp(root)
    root.mainloop()
